In [2]:
using LinearAlgebra
using SparseArrays
using CUDA, CUDA.CUBLAS, CUDA.CUSOLVER, CUDA.CUSPARSE

In [ ]:
A = Matrix{Float64}(undef, N, N)
b = Vector{Float64}(undef, N)

d_A = CuSparseMatrixCSC{Float64}(A)
d_b = CuArray{Float64}(b)

In [ ]:
A = Matrix{Float64}(undef, 128, 128)        # Almacenado en CPU

d_A = CuMatrix{Float64}(undef, 128, 128)    # Almacenado en la GPU

In [4]:
A = Matrix{Float64}(undef, 128, 128)    # Almacenado en CPU

A = inicializar_matriz()                # Rellenamos la matriz con valores        

d_A = CuMatrix{Float64}(A)              # Almacenado en la GPU

UndefVarError: UndefVarError: `rellenar_matriz` not defined

In [ ]:
d_A = CuMatrix{Float64}(undef, 128, 128)    # Asignamos memoria en la GPU

d_A = inicializar_matriz()                  # Llenamos la matriz en la GPU

A = Matrix{Float64}(d_A)                    # Asignamos memoria en la CPU y copiamos los valres desde la GPU.

In [ ]:
d_A = CuMatrix{Float64}(undef, 128, 128)    # Asignamos memoria en la GPU
d_b = CuVector{Float64}(undef, 128)         # Asignamos memoria en la GPU

d_A = inicializar_matriz()                  # Llenamos la matriz en la GPU
d_b = inicializar_vector()                  # Llenamos la vector en la GPU

d_C = d_A * d_b                             # Multiplicación matriz-vector
                                            # El resultado se guarda en la GPU

In [ ]:
function parallelsor(A::Matrix{Float64}, b::Vector{Float64}, ω = 1.2, ϵ = 1e-5)
    D = Diagonal(A)
    U = - triu(A,1)
    L = - tril(A,-1)

    DL = D-ω*L
    UD = ω*U+(1.0-ω)*D
    f = ω*b

    d_UD = CuSparseMatrixCSC{Float64}(UD)   # Tipo de dato CuSparseMatrixCSC
    d_f = CuVector{Float64}(f)

    d_A = CuSparseMatrixCSC{Float64}(A)
    d_b = CuVector{Float64}(b)
    d_x = CUDA.zeros(Float64, size(b))  # Asigna  memoria a una matriz y la inicializa con ceros

    while CUBLAS.norm(d_A*d_x - d_b) > ϵ    # Calcula la norma de un vector en la GPU
        g = Vector(d_UD*d_x + d_f)
        d_x = CuVector{Float64}(DL \ g)
    end
    return x = Vector(d_x)  # Solución aproximada
end